## Previsão de Séries Temporais 

In [1]:
# import packages
import yfinance as yf
import pandas as pd

In [2]:
# importar dados do Yahoo Finance
msft = yf.Ticker("^BVSP")
print(msft)
msft.info
data = msft.history(period="max")

yfinance.Ticker object <^BVSP>


In [3]:
# select columns
data = data[['Close']]

# transform in datetime
data.index = pd.to_datetime(data.index)

In [4]:
data = data[6000:6640]

In [5]:
# visu
import plotly.plotly as ply
import cufflinks as cf

data.iplot(title="IBOV")

In [6]:
# index to column
data.reset_index(inplace=True)
data

,Date,Close
0,2017-07-25,65668.00
1,2017-07-26,65011.00
2,2017-07-27,65277.00
3,2017-07-28,65497.00
4,2017-07-31,65920.00
...,...,...
635,2020-02-19,116518.00
636,2020-02-20,114586.00
637,2020-02-21,113681.00
638,2020-02-26,105718.00


### MODELAGEM

#### AUTOARIMA para selecionar parametros

In [19]:
from pyramid.arima import auto_arima

stepwise_model = auto_arima(data['Close'], 
                            trace=True, 
                            error_action='ignore', 
                            suppress_warnings=True,
                            start_p=1, start_q=1,
                            max_p=3, max_q=3, m=12,
                            start_P=0, seasonal=True,
                            d=1, D=1)
print(stepwise_model.aic())

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=10929.331, BIC=10951.535, Fit time=3.988 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=11025.609, BIC=11034.491, Fit time=0.079 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=10959.087, BIC=10976.851, Fit time=1.529 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=10908.973, BIC=10926.737, Fit time=5.519 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=10859.384, BIC=10881.589, Fit time=12.216 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 0, 12); AIC=10959.455, BIC=10977.219, Fit time=1.673 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=10869.965, BIC=10896.611, Fit time=33.722 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=11026.463, BIC=11039.786, Fit time=0.137 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(2, 1, 2, 12); AIC=10883.824, BIC=10914.911, Fit time=49.834 seconds
Fit ARI

### MODEL

In [39]:
prediction = pd.DataFrame(interval, predicted)

In [40]:
def addNullModel(data):
    data.reset_index(inplace=True)
    data.columns = ['Close', 'Date']
    data['Date'] = data['Date'].dt.to_period('d')
    return data 

prediction = pd.DataFrame(interval, predicted)
prediction = addNullModel(prediction)
prediction['Date'] = prediction['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [41]:
data

,Date,Close
0,2017-07-25,65668.00
1,2017-07-26,65011.00
2,2017-07-27,65277.00
3,2017-07-28,65497.00
4,2017-07-31,65920.00
...,...,...
635,2020-02-19,116518.00
636,2020-02-20,114586.00
637,2020-02-21,113681.00
638,2020-02-26,105718.00


In [30]:
#data['Date'] = data['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [43]:
# rename target columns and merge outer
datapred = pd.merge(data, prediction, how = 'outer', on = 'Date')
datapred.columns = ['date', 'valor original', 'valor predito']

In [45]:
def layLine(min, max):
    import plotly.graph_objs as go
    layout = go.Layout( {'legend': {'bgcolor': '#F5F6F9', 'font': {'color': '#4D5663'}},
                   'paper_bgcolor': '#F5F6F9',
                   'plot_bgcolor': '#F5F6F9',
                   'title': {'font': {'color': '#4D5663'}},
                   'xaxis': {'gridcolor': '#E1E5ED',
                             'showgrid': True,
                             'tickfont': {'color': '#4D5663'},
                             'title': {'font': {'color': '#4D5663'}, 'text': ''},
                             'zerolinecolor': '#E1E5ED'},
                   'yaxis': {'gridcolor': '#E1E5ED',
                             'range' : (min, max),
                             'showgrid': True,
                             'tickfont': {'color': '#4D5663'},
                             'title': {'font': {'color': '#4D5663'}, 'text': ''},
                             'zerolinecolor': '#E1E5ED'}}
    )
    return layout

In [77]:
# min and max on plot
mini = 40000
maxi = datapred['valor predito'].max()+(datapred['valor predito'].max()*0.4)
#datapred.set_index('date', inplace=True)

# definir layout
layout = layLine(mini, maxi)


In [78]:
# forecast visualization
datapred[['valor original', 'valor predito']].iplot(mode='lines',
                                   size = 8,
                                   colors=['#87cded', 'pink'],
                                   layout=layout.to_plotly_json(),
                                   filename='test')

In [56]:
dfprophet = data
dfprophet.columns = ['ds', 'y']

## o

In [51]:
from fbprophet import Prophet

In [57]:
prophet_model = Prophet()
prophet_model.fit(dfprophet)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [67]:
future = prophet_model.make_future_dataframe(periods=200)
predicted = prophet_model.predict(future)

In [61]:
predicted[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
835,2020-09-10,106297.490149,88457.833308,121232.674863
836,2020-09-11,106554.058644,89318.244685,122270.388230
837,2020-09-12,103019.156442,85225.014522,118874.587921
838,2020-09-13,103157.351397,85813.605516,118434.475210
839,2020-09-14,106884.024311,89188.646025,122820.148308


In [71]:
predicted

,ds,yhat
0,2017-07-25,68014.200439
1,2017-07-26,68041.548736
2,2017-07-27,68087.012924
3,2017-07-28,68238.264538
4,2017-07-31,68256.689236
...,...,...
835,2020-09-10,106297.490149
836,2020-09-11,106554.058644
837,2020-09-12,103019.156442
838,2020-09-13,103157.351397


In [68]:
predicted = predicted[['ds', 'yhat']]
#predicted.columns = ['Date', 'Close']
predicted['ds'] = predicted['ds'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [74]:
# rename target columns and merge outer
datapred = pd.merge(data, predicted, how = 'outer', on = 'ds')
datapred.columns = ['date', 'valor original', 'valor predito']

In [75]:
# forecast visualization
datapred[['valor original', 'valor predito']].iplot(mode='lines+markers',
                                   size = 8,
                                   colors=['#87cded', 'pink'],
                                   layout=layout.to_plotly_json(),
                                   filename='test')

## LSTM